In [8]:
import os
import pandas as pd
from openai import OpenAI
from func import get_user_watch_history

client = OpenAI()

merged_df = pd.read_csv("data/merged_df.csv")
unique_movie_titles = merged_df['title'].unique().tolist()

# def get_movies_and_predict_ratings():
user_response = input("Are you planning to watch the movie alone or with a partner? (alone[a]/partner[p]): ").strip().lower()
if user_response in ['alone', 'a']:
  user_id = int(input("Enter Your User ID:"))
  if user_id in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a movie recommender system that will compare user previous watch history and ratings.Based on the genre and ratings of those movies that this particular user gave, please recommend 10 movie that this user would enjoy watching and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id}'s previously watch movies include {titles_list} and the corresponded ratings are {ratings_list}. Please suggest 10 movie based on the watch history and predict ratings of those 10 movies based on user preferences"}
    ]
  )
  else:
    print("This is not a valid user ID, please try again")
elif user_response in ['partner', 'p']:
  user_A = int(input("Enter Your User ID:"))
  user_B = int(input("Enter Your Partner's User ID:"))
  if user_A and user_B in merged_df['userId'].values:
    titles_list_A, ratings_list_A = get_user_watch_history(user_A, merged_df)
    titles_list_B, ratings_list_B = get_user_watch_history(user_B, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a movie recommender system that will compare user's previous watch history and ratings. Based on the genre and ratings of movies from 2 users, please recommend 10 movie that these 2 users would enjoy watching together and predict the rating of these 10 movies"},
      {"role": "user", "content": f"User {user_A}'s previously watch movies include {titles_list_A} and the corresponded ratings are {ratings_list_A}. User {user_B}'s previously watch movies include {titles_list_B} and the corresponded ratings are {ratings_list_B} Please suggest 10 movie that both users would enjoy watching based on the watch histories and predict specific ratings of those 10 movies based on user preferences"}
    ]
  )
else:
    print("Invalid response. Please answer with 'alone','a' or 'partner','p'.")
    

print(completion.choices[0].message) 

ChatCompletionMessage(content="Based on User 165's previously watched movies and ratings, we can identify the genres and movies that this user enjoys. Here are 10 movie recommendations for User 165:\n\n1. The Shawshank Redemption (1994) - Drama\n2. Inception (2010) - Sci-Fi/Thriller\n3. The Departed (2006) - Crime/Drama\n4. The Dark Knight (2008) - Action/Crime/Drama\n5. Fight Club (1999) - Drama/Thriller\n6. Pulp Fiction (1994) - Crime/Drama\n7. The Social Network (2010) - Biography/Drama\n8. The Prestige (2006) - Drama/Mystery/Thriller\n9. Eternal Sunshine of the Spotless Mind (2004) - Drama/Romance/Sci-Fi\n10. The Matrix (1999) - Action/Sci-Fi\n\nNow, let's predict the ratings for these 10 movie recommendations based on User 165's preferences. Please note that the predicted ratings are based on the assumption that User 165 will continue to rate movies similarly to their previous ratings:\n\n1. The Shawshank Redemption (1994) - Predicted Rating: 4.0\n2. Inception (2010) - Predicted R

# Watch party with 2 userID
user_A = 55

user_B = 3

In [5]:
print(completion.choices[0].message)

ChatCompletionMessage(content="To recommend movies that both User 55 and User 3 would enjoy watching together, I will analyze their watch histories and ratings. Based on their common movie interests, I will suggest 10 movies that are likely to appeal to both users. To predict the ratings, I will look at their previous ratings for similar movies and make an estimate. \n\nHere are 10 movie recommendations for User 55 and User 3:\n\n1. Highlander (1986)\n2. Escape from L.A. (1996)\n3. Three Kings (1999)\n4. Field of Dreams (1989)\n5. Lock, Stock & Two Smoking Barrels (1998)\n6. Conan the Barbarian (1982)\n7. Thomas Crown Affair, The (1999)\n8. Road Warrior, The (Mad Max 2) (1981)\n9. Snow Dogs (2002)\n10. Snatch (2000)\n\nNow, let's predict the ratings for these movies based on their previous preferences:\n\n1. Highlander (1986) - User 55: 4.0, User 3: 3.5\n2. Escape from L.A. (1996) - User 55: 3.5, User 3: 4.5\n3. Three Kings (1999) - User 55: 4.0, User 3: 0.5\n4. Field of Dreams (1989) 

# Recommend based on genre

In [13]:
from func import find_top_3_genre

# def get_movie_genre():
user_id_genre = int(input("We will suggest movies based on your favorite genre, please enter your user ID:"))
if user_id_genre in merged_df['userId'].values:
    top_3_genre = find_top_3_genre(user_id_genre,merged_df)
    genre_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a movie recommender system that will recommend 10 movie in this user's favorite genres and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id_genre}'s favorite genre include {top_3_genre}. Please suggest 10 movie in these genre for the user"}
    ]
)
else:
    print("This is not a valid user ID, please try again")

print(genre_response.choices[0].message)

ChatCompletionMessage(content="Based on User 234's favorite genres, here are 10 movie recommendations:\n\n1. Interstellar (IMAX)\n2. A Quiet Place (Horror)\n3. The Conjuring (Horror)\n4. The Phantom of the Opera (Musical)\n5. La La Land (Musical)\n6. Les Misérables (Musical)\n7. Dunkirk (IMAX)\n8. Get Out (Horror)\n9. The Greatest Showman (Musical)\n10. Gravity (IMAX)\n\nThese movies cover a range of genres including IMAX, Horror, and Musical, which are among User 234's favorites. Enjoy watching!", role='assistant', function_call=None, tool_calls=None)


# Recommend based on similar user

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similar_users(user_id, data):
    user_movie_matrix = data.pivot_table(index='userId', columns='title', values='rating')
    user_movie_matrix = user_movie_matrix.fillna(0)
    similarity_matrix = cosine_similarity(user_movie_matrix)
    similarity_df = pd.DataFrame(similarity_matrix, index=user_movie_matrix.index, columns=user_movie_matrix.index)
    similar_users = similarity_df[user_id].sort_values(ascending=False)
    most_similar = similar_users.index[1]
    return most_similar

In [47]:
#from func import get_similar_users

# def rec_from_similar_user():
user_id_sim = int(input("We will suggest movies you may also like based on similar user, please enter your user ID:"))
if user_id_sim in merged_df['userId'].values:
    most_similar = get_similar_users(user_id_sim, merged_df)
    titles_list, ratings_list = get_user_watch_history(most_similar, merged_df)
    response_sim = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "You are a movie recommender system that will suggest movies that the user may also like based on similar user's watch histories. From the most similar user's watch histories, please recommend 10 movies that this user would enjoy watching and predict the rating of these 10 movies given by this user"},
    {"role": "user", "content": f"User {user_id_sim}'s most similar user is user {most_similar}, which has previously watched {titles_list} and the corresponded ratings are {ratings_list}. Please suggest 10 movies that user {user_id_sim} may also like and provide rating prediction of those 10 movies based on user preferences"}
    ]
)
else:
    print("This is not a valid user ID, please try again")   

print(response_sim.choices[0].message)

ChatCompletionMessage(content="Based on the watch histories of the most similar user (user 46), here are 10 movie recommendations for user 134 along with predicted ratings:\n\n1. Pulp Fiction (1994) - Predicted Rating: 4.5\n2. The Shawshank Redemption (1994) - Predicted Rating: 4.5\n3. The Lion King (1994) - Predicted Rating: 4.0\n4. Fight Club (1999) - Predicted Rating: 4.5\n5. The Matrix (1999) - Predicted Rating: 4.5\n6. Forrest Gump (1994) - Predicted Rating: 4.0\n7. The Dark Knight (2008) - Predicted Rating: 4.5\n8. Inception (2010) - Predicted Rating: 4.5\n9. Goodfellas (1990) - Predicted Rating: 4.0\n10. The Lord of the Rings: The Fellowship of the Ring (2001) - Predicted Rating: 4.0\n\nPlease note that these ratings are just predictions based on similar user preferences and may not accurately reflect user 134's actual ratings.", role='assistant', function_call=None, tool_calls=None)
